In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.musique.qa import answer_question_standard
from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark

set_seed(89)

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
from bellek.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)

print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [5]:
df['paragraphs'] = df['paragraphs'].map(lambda ps: [p for p in ps if p['is_supporting']])

In [6]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [7]:
qa_func = answer_question_standard

In [8]:
results = []

In [9]:
completion_params = {
    "temperature": 0.1
}

In [10]:
def replace_paragraphs_with_offline_triplets(df, jerx_file):
    df = df.copy()

    jerx_df = pd.read_json(jerx_file, lines=True)
    jerx_mapping = {(row['id'], row['paragraph_idx']): row['generation'] for _, row in jerx_df.iterrows()}

    def replace_paragraphs(row):
        new_paragraphs = []
        for p in row['paragraphs']:
            p = deepcopy(p) 
            triplets_str = jerx_mapping[(row['id'], p['idx'])].strip()
            p['paragraph_text'] = triplets_str
            new_paragraphs.append(p)
        row['paragraphs'] = new_paragraphs
        return row

    return df.progress_apply(replace_paragraphs, axis=1) 

## llama-sft

In [12]:
jerx_inferences_path = Path('/Users/bdsaglam/dev/repos/bellek/data/raw/musique-evaluation/jerx-inferences/llama3-sft-aw7ihmbc.jsonl')
df_llama_sft = replace_paragraphs_with_offline_triplets(df, jerx_inferences_path)
df_llama_sft.head()

  0%|          | 0/100 [00:00<?, ?it/s]

KeyError: ('2hop__263638_69048', 5)

In [19]:
for i in range(1,4):
    df_llama_sft, scores = benchmark(df_llama_sft, qa_func, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-sft", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.55,
  "f1": 0.6413358863358863,
  "fuzzy_match": 0.66
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.55,
  "f1": 0.6413358863358863,
  "fuzzy_match": 0.66
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.55,
  "f1": 0.6413358863358863,
  "fuzzy_match": 0.66
}


# Report

In [22]:
pd.options.display.float_format = '{:,.3f}'.format

In [23]:
report_df = pd.DataFrame.from_records(results, columns=['run', 'context', 'retrieval', 'jerx', 'exact_match', 'fuzzy_match', 'f1'])
report_df.drop(columns=['context', 'retrieval', 'fuzzy_match'])

,run,jerx,exact_match,f1
0,1,llama-zero-shot,0.530,0.624
1,2,llama-zero-shot,0.530,0.624
2,3,llama-zero-shot,0.530,0.624
3,1,llama-few-shot,0.530,0.651
4,2,llama-few-shot,0.530,0.651
5,3,llama-few-shot,0.530,0.651
6,1,llama-sft,0.550,0.641
7,2,llama-sft,0.550,0.641
8,3,llama-sft,0.550,0.641


In [25]:
report_df[['jerx', 'exact_match', 'f1']].groupby('jerx').mean().loc[['llama-zero-shot', 'llama-few-shot', 'llama-sft']]

,exact_match,f1
jerx,,
llama-zero-shot,0.530,0.624
llama-few-shot,0.530,0.651
llama-sft,0.550,0.641
